In [ ]:
import os
import json
from json import JSONDecodeError
import numpy as np
from tqdm import tqdm
import torch

from collections import defaultdict

from util.constants import Topic

In [ ]:
# Models
from sentence_transformers import SentenceTransformer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens').to(device)

def model(titles):
    return sbert_model.encode(titles)

Get latents for all videos

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
RESULTS_DIR = os.path.join("..", "data", "title-latents")

video_results_dir = os.path.join("..","..","DATA","title-latents","videos")
channel_results_dir = os.path.join(RESULTS_DIR, "channels")

if not os.path.exists(video_results_dir):
    os.makedirs(video_results_dir)

def get_done_list(dir):
    return [nm.replace(".json",'').replace('.pt','') for nm in os.listdir(dir)]

In [ ]:
# Run the code in batches
for channel,videos in tqdm(channel_videos_dict.items()):
    done_path = os.path.join(RESULTS_DIR, "videos_done", channel+".json")
    ids = [vid["id"] for vid in videos[:30]]
    try:
        with open(done_path, "r") as f:
            done_ids = json.load(f)
        if set(ids).issubset(done_ids): # Every id is done
            continue
    except FileNotFoundError:
        pass

    titles = [vid["title"] for vid in videos[:30]]
    
    batch_latents = np.array(model(titles))

    path = os.path.join(video_results_dir, f"{channel}.pt")
    torch.save(batch_latents, path)
    with open(done_path, "w") as f:
        json.dump(ids, f)

    torch.cuda.empty_cache()

Channel stats

In [ ]:
# Read data
channel_videos_dict = {}
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        channel_videos_dict.update(json.load(f))

In [ ]:
# Calculate channel results
for channel,videos in tqdm(channel_videos_dict.items()):
    in_path = os.path.join(video_results_dir,channel+".pt")
    result_list = torch.load(in_path)

    channel_mean = result_list.mean(axis=0)
    channel_result = {
        "std": float(result_list.std()),
        "len": len(result_list),
    }

    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    with open(filepath, "w") as f:
        json.dump(channel_result, f)
    filepath = os.path.join(RESULTS_DIR, "channels_mean", f"{channel}.pt")
    torch.save(channel_mean, filepath)

Category stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "channel2category.json"), "r") as f:
    channel2cat = json.load(f)

In [ ]:
# Make list of results per channel for each category
category_results_list = defaultdict(list)
for channel in tqdm(get_done_list(channel_results_dir)):
    cat = channel2cat[channel]
    results = {}
    for folder,ext in [("channels",".json"), ("channels_mean",".pt")]:
        filepath = os.path.join(RESULTS_DIR, folder, channel+ext)
        try:
            if ext == ".json":
                with open(filepath, "r") as f:
                    results.update(json.load(f))
            elif ext == ".pt":
                results.update({"mean": torch.load(filepath)})
        except JSONDecodeError:
            print(f"couldn't open {channel}; deleting file")
            os.remove(filepath)
    category_results_list[cat].append(results)

In [ ]:
# Calculate category results
for cat,stats_list in category_results_list.items():
    mean_list = np.array([channel_stats["mean"] for channel_stats in stats_list])
    std_list = np.array([channel_stats["std"] for channel_stats in stats_list])
    category_mean = mean_list.mean(axis=0)
    category_result = {
        "std": float(mean_list.std()),
        "avg_std_of_channels": std_list.mean(),
        "len": len(mean_list),
    }

    filepath = os.path.join(RESULTS_DIR, "categories", f"{cat}.json")
    with open(filepath, "w") as f:
        json.dump(category_result, f)
    filepath = os.path.join(RESULTS_DIR, "categories_mean", f"{cat}.pt")
    torch.save(category_mean, filepath)